In [1]:
# conda: holo37

# cd to the 'lib' directory in your notebooks directory
# $ export LIBROOT=`pwd`
# Start jupyter lab from any directory

import sys, os
if (os.environ.get('LIBROOT')):
    sys.path.append(os.environ.get('LIBROOT'))


# import gridUtils.py from local folder via LIBROOT path specified above
from gridutils import GridUtils
import numpy as np
import cartopy.crs as ccrs
import panel as pn
import os, sys
import cartopy
import matplotlib.pyplot as plt
import netCDF4 as nc
import warnings
import xarray as xr
#import xgcm
from io import BytesIO
warnings.filterwarnings('ignore')

pn.extension()

# aspect 4:3, default dpi=144
widthIn = 5.0
heightIn = (widthIn * 3.0) / 4.0
defaultPlotFigureSize = (widthIn, heightIn)

In [2]:


# Declare panel options
projNames = ["Mercator", "Lambert Conformal Conic", "Polar Stereographic"]
projNamesGridTools = {
    'Mercator': 'Mercator',
    'Lambert Conformal Conic': 'LambertConformalConic',
    'Polar Stereographic': 'NorthPolarStereo'
}
projCarto = [ccrs.Mercator(), ccrs.LambertConformal(),  ccrs.NorthPolarStereo()]
projDict = dict(zip(projNames,projCarto))
projSel = pn.widgets.Select(name='Grid File Projection', options=projNames, value=projNames[1])
projNamesGridTools = {
    'Mercator': 'Mercator',
    'Lambert Conformal Conic': 'LambertConformalConic',
    'Polar Stereographic': 'NorthPolarStereo'
}

gridRes = pn.widgets.Spinner(name="Gird Resolution", value=1, step=1, start=0, end=10, width=90)
gridMode = pn.widgets.Spinner(name="Grid Mode", value=2, step=1, start=0, end=10, width=90)
unitNames = ['degrees','meters','points']
dxdyUnits = pn.widgets.Select(name='Units', options=unitNames, value=unitNames[0])
dx = pn.widgets.Spinner(name="dx", value=30, step=1, start=0, end=100, width=90)
dy = pn.widgets.Spinner(name="dy", value=30, step=1, start=0, end=100, width=90)
lon0 = pn.widgets.Spinner(name="lon0 (0 to 360)", value=300, step=1, start=0, end=360, width=90)
lat0 = pn.widgets.Spinner(name="lat0 (-90 to 90)", value=40, step=1, start=-90, end=90, width=90)
tilt = pn.widgets.Spinner(name="Tilt (-90 to 90)", value=30, step=1, start=-90, end=90, width=90)
plotProjection = pn.widgets.Select(name='Projection', options=projNames, value=projNames[1])

statusWidget = pn.widgets.TextInput(name='Information', value="", background="skyblue")

In [3]:
# Panel functions
def make_grid(lon0,dx,lat0,dy,tilt, projSel, gridRes, gridMode, dxdyUnits):
    grd.clearGrid()
    #refineR   = 1   # Inverse of nominal resolution
    #refineS   = 2   # Set to 2 for supergrid
    # Specify the grid parameters
    # nominalSpacing should be 2.0 for supergrid
    grd.setGridParameters({
        'projection': {
            'name': projSel,
            'lon_0': lon0,
            'lat_0': lat0
        },
        'dx': lon_span,
        'dy': lat_span,
        'dxUnits': 'degrees',
        'dyUnits': 'degrees',    
        'gridResolution': nomRes,
        'gridMode': nomSpace,
        'tilt': tilt
    })
    grd.makeGrid()
    
    return

def make_plot(lon0,dx,lat0,dy,tilt, projSel, gridRes, gridMode, dxdyUnits):
    width=800
    height=600
    grd.clearGrid()
    mp_title = "Nearside Perspective: " + str(dx) + "x" + str(dy) + " with " + str(tilt) + " degree tilt"

    grd.setGridParameters({
        'projection': {
            'name': projNamesGridTools[projSel],
            'lon_0': lon0,
            'lat_0': lat0
        },
        'dx': dx,
        'dy': dy,
        'dxUnits': 'degrees',
        'dyUnits': 'degrees',
        'gridResolution': gridRes,
        'gridMode': gridMode,
        'tilt': tilt
    })
    grd.makeGrid()
    
    grd.setPlotParameters(
        {
            'figsize': defaultPlotFigureSize,
            'projection' : {
                'name': 'NearsidePerspective',
                'lat_0': lat0,
                'lon_0': lon0
            },
            'extent': [lon0 - dx, lon0 + dx, lat0 - dy, lat0 + dy],
            'iLinewidth': 1.0,
            'jLinewidth': 1.0,
            'showGridCells': True,
            'title': mp_title,
            'iColor': 'k',
            'jColor': 'k'
        }
    )
    
    (figure, axes) = grd.plotGrid()
    # Suppress showing figure in a notebook cell outside the panel/bokeh/geoview application.
    # This still allows the figure object to be passed around and to the application.
    plt.close()
    return figure

def test_grid(lon0,dx,lat0,dy,tilt, projSel, gridRes, gridMode, dxdyUnits):
    mom6_grid = xr.open_dataset("/home/cermak/gridtools/gridTools/configs/test/mom6_xgcm_grid.nc")
    return mom6_grid

def update_(event):
    plotWindow.object = make_plot(
        float(lon0.value), float(dx.value), float(lat0.value), float(dy.value), 
        float(tilt.value), plotProjection.value, 
        float(gridResolution.value), float(gridMode.value),
        dxdyUnits.value)

def download_cb():
    bout = test_grid(lon0,dx,lat0,dy,tilt, projSel, gridRes, gridMode, dxdyUnits).to_netcdf()
    bio = BytesIO()
    bio.write(bout)
    bio.seek(0)
    return bio

# need to have plotwindow[1].object, not plotwindow.object - not sure as to why...
def update_plot(event):
    plotwindow[1].object = make_plot(lon0.value,dx.value,lat0.value,dy.value,tilt.value, projSel.value, gridRes.value, gridMode.value, dxdyUnits)


In [4]:
# Initialization

# This needs to exist prior to calling make_plot()
grd = GridUtils()



In [5]:
# create file download button
gridDownload = pn.widgets.FileDownload(
    callback=download_cb,
    button_type='success',
    filename='gridFile.nc')


plotUpdateButton = pn.widgets.Button(name='Refresh Plot', button_type='primary')
plotUpdateButton.on_click(update_plot)


#header = pn.Row(pn.panel(mom6_logo, width=170),  pn.layout.Spacer(width=10), 
#                pn.Column(pn.Pane(title1, width=1000), pn.Pane(instruction, width=1000)))




sel_box = pn.WidgetBox(pn.Column("## Visualize Grid"), lon0,dx,lat0,dy,tilt, gridRes, gridMode, dxdyUnits, plotUpdateButton,pn.layout.Spacer(height=20), pn.Column("## Download Grid File"), projSel, gridDownload)
plotwindow = pn.Row(sel_box, pn.Column(pn.pane.Matplotlib(make_plot(lon0.value,dx.value,lat0.value,dy.value,tilt.value, projSel.value, gridRes.value, gridMode.value, dxdyUnits),width=800, height=600), statusWidget),
                    test_grid(lon0.value,dx.value,lat0.value,dy.value,tilt.value, projSel.value, gridRes.value, gridMode.value, dxdyUnits))



# Attach application to gridUtils for integration into panel, etc
# Do this just before launching the application
grd.application(
    app={
        'messages': statusWidget,
        'defaultFigureSize': defaultPlotFigureSize
    }
)



dashboard = pn.Column(plotwindow)
#dashboard.show()
dashboard.app(notebook_url="192.168.131.54:8888")